In [ ]:
#hide
#default_exp model_analysis
from nbdev.showdoc import *

# Fitting Models to Data

A key step for evaluating any cognitive model is to find and measure model fit to real behavioral datasets.
This process requires a few new sections of code. These include:

1. A `prepare_data` function that loads data from a provided path and formats it to support both efficient
   model fitting and data visualization.
2. A `likelihood` function that computes the likelihood of the data given a model and a specified parameter
   configuration.
3. A `likelihood_search` function that searches for the parameter configuration of a specified model that
   makes the provided data the most likely.
4. A `visualize_fit` function that visually compares a selected dataset with a model fitted to it using
   figures such as the serial position and conditional recall probability curve.
5. We may also need to try optimizing `models.InstanceCMR` to help speed up fitting, which can be a
   pretty time-consuming process!

We will use this code to fit our updated model to data sourced for a classical 1962 study reporting the
serial position effect of free recall, cited below. To confirm that our discovered parameter configuration
really works, we'll visualize the resulting fit. We'll also add some tests confirming the robustness of the
fitting algorithm.

> Murdock, B. B., Jr. (1962). The serial position effect of free recall. Journal of Experimental Psychology,
> 64(5), 482-488. https://doi.org/10.1037/h0045106  

## Data Preparation

To decide on how we'll make a function for data preparation, we'll consider the dataset under current interest at
`data/MurdData_clean.mat` and hope that other relevant datasets will have a similar structure. The code is thus
tentative, since using other datasets or using the current dataset for other purposes may require updating or
expanding this code.

It has three `LL` structures that each seem to correspond to a different data set with different list lengths.  Inside
each structure is:
- `recalls` with 1200 rows and 50 columns. Each row presumably represents a subject, and each column seems to
  correspond to a recall position, with -1 coded for intrusions. `MurdData_clean.mat` probably doesn't have these
  intrusions coded at all.
- `listlength` is an integer indicating how long the studied list is.
- `subject` is a 1200x1 vector coding the identities of each subject for each row. Each subject seems to get 80 rows a
  piece. He really got that much data for each subject? 
- `session` similarly codes the index of the session under consideration, and it's always 1 in this case.
- `presitemnumbers` probably codes the number associated with each item. Is just its presentation index.

At first, we will only work with on `LL` structure at a time.

We need a few structures extracted from a selected dataset.
1. A `trials` array, where each row identifies a unique trial of responses and each call corresponds to a unique
   recall index. Entries are 0 where no item is recalled, -1 where an inapplicable item is recalled, while other
   entries are 1-indexed based on the order in which they were presented in the list.
2. A "long' format table representing recall data as specified in the [`psifr`
   documentation](https://psifr.readthedocs.io/en/latest/guide/import.html).

In [ ]:
#export
#hide
import scipy.io as sio
import numpy as np
import pandas as pd
from psifr import fr

def prepare_murddata(path, dataset_index):
    """
    Prepares data formatted like `data/MurdData_clean.mat` for fitting.

    Loads data from `path` with same format as `data/MurdData_clean.mat` and returns a selected dataset as an array of
    unique recall trials and a dataframe of unique study and recall events organized according to `psifr`
    specifications.

    **Arguments**:  
    - path: source of data file  
    - dataset_index: index of the dataset to be extracted from the file

    **Returns**:
    - trials: int64-array where rows identify a unique trial of responses and columns corresponds to a unique recall
        index.  
    - merged: as a long format table where each row describes one study or recall event.  
    - list_length: length of lists studied in the considered dataset
    """
    # load all the data
    matfile = sio.loadmat(path, squeeze_me=True)
    murd_data = [matfile['data'].item()[0][i].item() for i in range(3)]
    
    # encode dataset into psifr format
    trials, list_length, subjects = murd_data[dataset_index][:3]
    trials = trials.astype('int64')
    
    data = []
    for trial_index, trial in enumerate(trials):

        # every time the subject changes, reset list_index
        if not data or data[-1][0] != subjects[trial_index]:
            list_index = 0
        list_index += 1

        # add study events
        for i in range(list_length):
            data += [[subjects[trial_index], 
                      list_index, 'study', i+1, i+1]]

        # add recall events
        for recall_index, recall_event in enumerate(trial):
            if recall_event != 0:
                data += [[subjects[trial_index], list_index, 
                          'recall', recall_index+1, recall_event]]

    data = pd.DataFrame(data, columns=[
        'subject', 'list', 'trial_type', 'position', 'item'])
    merged = fr.merge_free_recall(data)
    return trials, merged, list_length

In [ ]:
try:
    show_doc(prepare_murddata, title_level=3)
except:
    pass

We can generate a quick preview of some datasets using this function.

In [ ]:
murd_trials, murd_events, murd_length = prepare_murddata('../data/MurdData_clean.mat', 0)

murd_events.head()

|    |   subject |   list |   item |   input |   output | study   | recall   |   repeat | intrusion   |
|---:|----------:|-------:|-------:|--------:|---------:|:--------|:---------|---------:|:------------|
|  0 |         1 |      1 |      1 |       1 |        5 | True    | True     |        0 | False       |
|  1 |         1 |      1 |      2 |       2 |        7 | True    | True     |        0 | False       |
|  2 |         1 |      1 |      3 |       3 |      nan | True    | False    |        0 | False       |
|  3 |         1 |      1 |      4 |       4 |      nan | True    | False    |        0 | False       |
|  4 |         1 |      1 |      5 |       5 |      nan | True    | False    |        0 | False       |

## Configuring the Parameter Search
To fit the model to some dataset, we select a cost function that scales against the likelihood that the model
with a specified parameter configuration could have generated the specified dataset. Here we'll use those built just for InstanceCMR.

In [ ]:
from instance_cmr.models import *

In [ ]:
try:
    show_doc(icmr_likelihood, title_level=3)
except:
    pass

We return a sum of negative log likelihoods because our fitting functions search for parameters that minimize
error functions and log-likelihoods are negative. For example,

In [ ]:

lb = np.finfo(float).eps
hand_fit_parameters = {
    'item_count': murd_length,
    'encoding_drift_rate': .8,
    'start_drift_rate': .7,
    'recall_drift_rate': .8,
    'shared_support': 0.01,
    'item_support': 1.0,
    'learning_rate': .3,
    'primacy_scale': 1,
    'primacy_decay': 1,
    'stop_probability_scale': 0.01,
    'stop_probability_growth': 0.3,
    'choice_sensitivity': 2
}
icmr_likelihood(murd_trials[:60], **hand_fit_parameters)

returns `1154.6050515722645`.

When it comes to model fitting, we will normally wrap this loss function within another that fixes function
parameters we aren't seeking a fit for - for example, our data structure `trials`. In general, our true
objective function should fill free parameters of `icmr_likelihood` using values in a single `x` array.

In [ ]:
try:
    show_doc(icmr_objective_function, title_level=3)
except:
    pass

We can generate and apply the function to compute loss for different configurations of just
`encoding_drift_rate` with code like the following:

In [ ]:
cost_function = icmr_objective_function(murd_trials[:60], hand_fit_parameters, ['encoding_drift_rate'], )

cost_function([.8]), cost_function([.3])

Which returns (again) `1154.6050515722645`, and `1338.7420002341266`.

From here, the function searching the parameter space to find model configurations that fit as well to data
as possible is mostly written for us. We use the `differential_evolution` function from `scipy.optimize`. The
entire function specification can be found in [the corresponding
docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html), but
its main requirements are a cost function to be minimized and a `bounds` array specifying in each row a (min,
max) pair constraining search over each parameter. With the cost function generated above, we can find the
best fitting value of `encoding_drift_rate` to `murd_trials[:60]` for our `InstanceCMR` class very
efficiently after specifying a bounds array that constrains search between 0 and 1:

In [ ]:
from scipy.optimize import differential_evolution

result = differential_evolution(cost_function, [(np.finfo(float).eps, 1-np.finfo(float).eps)], disp=True)
result

This function returns an output with the following attributes:

```
     fun: 1153.963638767822
     jac: array([0.])
 message: 'Optimization terminated successfully.'
    nfev: 57
     nit: 2
 success: True
       x: array([0.81981498])
```

The `x` attribute of the result object contains the best parameter configuration found, while the `fun`
attribute represents the overall cost of the configuration as computed with our specified cost function. 

## Results
We can visually compare the behavior of the model with these parameters against the data it's fitted to with a new
`visualize_fit` function.

In [ ]:
# export
# hide
import seaborn as sns
import matplotlib.pyplot as plt

def visualize_fit(model_class, parameters, data, data_query=None, experiment_count=1000, savefig=False):
    """
    Apply organizational analyses to visually compare the behavior of the model with these parameters against
    specified dataset.
    """
    
    # generate simulation data from model
    model = model_class(**parameters)
    try:
        model.experience(np.eye(model.item_count, model.item_count + 1, 1))
    except ValueError:
        model.experience(np.eye(model.item_count, model.item_count))
    sim = []
    for experiment in range(experiment_count):
        sim += [[experiment, 0, 'study', i + 1, i] for i in range(model.item_count)]
    for experiment in range(experiment_count):
        sim += [[experiment, 0, 'recall', i + 1, o] for i, o in enumerate(model.free_recall())]
    sim = pd.DataFrame(sim, columns=['subject', 'list', 'trial_type', 'position', 'item'])
    sim_data = fr.merge_free_recall(sim)
    
    # generate simulation-based spc, pnr, lag_crp
    sim_spc = fr.spc(sim_data).reset_index()
    sim_pfr = fr.pnr(sim_data).query('output <= 1') .reset_index()
    sim_lag_crp = fr.lag_crp(sim_data).reset_index()
    
    # generate data-based spc, pnr, lag_crp
    data_spc = fr.spc(data).query(data_query).reset_index()
    data_pfr = fr.pnr(data).query('output <= 1').query(data_query).reset_index()
    data_lag_crp = fr.lag_crp(data).query(data_query).reset_index()
    
    # combine representations
    data_spc['Source'] = 'Data'
    sim_spc['Source'] = model_class.__name__
    combined_spc = pd.concat([data_spc, sim_spc], axis=0)
    
    data_pfr['Source'] = 'Data'
    sim_pfr['Source'] = model_class.__name__
    combined_pfr = pd.concat([data_pfr, sim_pfr], axis=0)
    
    data_lag_crp['Source'] = 'Data'
    sim_lag_crp['Source'] = model_class.__name__
    combined_lag_crp = pd.concat([data_lag_crp, sim_lag_crp], axis=0)
    
    # generate plots of result
    # spc
    g = sns.FacetGrid(dropna=False, data=combined_spc)
    g.map_dataframe(sns.lineplot, x='input', y='recall', hue='Source')
    g.set_xlabels('Serial position')
    g.set_ylabels('Recall probability')
    plt.title('P(Recall) by Serial Position Curve')
    g.add_legend()
    g.set(ylim=(0, 1))
    if savefig:
        plt.savefig('figures/{}_fit_spc.jpeg'.format(model_class.__name__), bbox_inches='tight')
    else:
        plt.show()
    
    #pdf
    h = sns.FacetGrid(dropna=False, data=combined_pfr)
    h.map_dataframe(sns.lineplot, x='input', y='prob', hue='Source')
    h.set_xlabels('Serial position')
    h.set_ylabels('Probability of First Recall')
    plt.title('P(First Recall) by Serial Position')
    h.add_legend()
    h.set(ylim=(0, 1))
    if savefig:
        plt.savefig('figures/{}_fit_pfr.jpeg'.format(model_class.__name__), bbox_inches='tight')
    else:
        plt.show()
    
    # lag crp
    max_lag = 5
    filt_neg = f'{-max_lag} <= lag < 0'
    filt_pos = f'0 < lag <= {max_lag}'
    i = sns.FacetGrid(dropna=False, data=combined_lag_crp)
    i.map_dataframe(
        lambda data, **kws: sns.lineplot(data=data.query(filt_neg),
                                         x='lag', y='prob', hue='Source', **kws))
    i.map_dataframe(
        lambda data, **kws: sns.lineplot(data=data.query(filt_pos),
                                         x='lag', y='prob', hue='Source', **kws))
    i.set_xlabels('Lag')
    i.set_ylabels('Recall Probability')
    plt.title('Recall Probability by Item Lag')
    i.add_legend()
    i.set(ylim=(0, 1))
    if savefig:
        plt.savefig('figures/{}_fit_crp.jpeg'.format(model_class.__name__), bbox_inches='tight')
    else:
        plt.show()

In [ ]:
try:
    show_doc(visualize_fit, title_level=3)
except:
    pass

With the function we can plot the model with the resulting parameter configuration against the actual data in
one line:

In [ ]:
visualize_fit(InstanceCMR, {**hand_fit_parameters, **{'encoding_drift_rate': result.x[0]}}, murd_events, 'subject == 1', experiment_count=1000, savefig=True)

Which results in figures like these:

![](figures/fit_spc.jpeg)

![](figures/fit_pfr.jpeg)

![](figures/fit_crp.jpeg)

These fits can be enhanced by freeing all model parameter for optimization, but the process takes too long to
occur in the context of this notebook.